In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd, os
import time
from urllib.parse import urljoin, urlparse
from time import sleep

In [2]:
empresas_logistica = {
    "Correos": "https://www.correos.es",
    "DHL": "https://www.dhl.es",
    "SEUR": "https://www.seur.com",
    "MRW": "https://www.mrw.es",
    "FedEx": "https://www.fedex.com/es-es/home.html",
    "UPS": "https://www.ups.com/es/es/Home.page",
    "GLS": "https://gls-group.eu/ES/es/home",
    "NACEX": "https://www.nacex.es",
    "Amazon Logistics": "https://logistics.amazon.es"
}


In [3]:
def obtener_urls(base_url, delay=1):
    """
Descarga la página principal y extrae las URLs internas
Acepta URLs del dominio principal y subdominios
"""
    try:
        # Realiza la petición HTTP a la URL de la enpresa con 10s máximos de espera
        response = requests.get(base_url, timeout=10)
        
        # Parsea el HTML recibido
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtiene el dominio principal
        dominio_empresa = urlparse(base_url).netloc

        # Elimina "www." al inicio para igualar subdominios y dominio principal

        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Set para almacenar URLs únicas
        urls = set()

        # Itera por todos los enlaces (a, href)
        for link in soup.find_all('a', href=True):
            href = link['href'] # Extrae el href del enlace
            href = urljoin(base_url, href) #Convierte relativo a absoluto

            #Obtiene el dominio del enlace
            dominio_href = urlparse(href).netloc
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añade la URL solo si es del dominio principal o subdomino
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Devuelve las URLs únicas como lista
        return list(urls)

    except Exception as e:
        print(f'Error al acceder a {base_url}: {e}')
        return []
 

        

        

    

In [4]:
resultados = []
for nombre, url_base in empresas_logistica.items():
    print(f'Scraping empresa: {nombre}')
    urls_empresa = obtener_urls(url_base) 
    for url in urls_empresa:
        resultados.append({'empresa': nombre, 'url': url})
    time.sleep(2)
    

Scraping empresa: Correos
Scraping empresa: DHL
Scraping empresa: SEUR
Scraping empresa: MRW
Scraping empresa: FedEx
Scraping empresa: UPS
Error al acceder a https://www.ups.com/es/es/Home.page: HTTPSConnectionPool(host='www.ups.com', port=443): Read timed out. (read timeout=10)
Scraping empresa: GLS
Scraping empresa: NACEX
Scraping empresa: Amazon Logistics


In [6]:
# Convertirmos los resultados en un DataFrame
df_scrap = pd.DataFrame(resultados)
#Guardamos el DataFrame en un archivo CSV
df_scrap.to_csv('../data/raw/logistica_legitimas_crudo.csv', index=False)

In [7]:
print(df_scrap['empresa'].value_counts())




empresa
GLS      95
NACEX    85
MRW      75
SEUR     62
DHL      48
Name: count, dtype: int64
